In [ ]:
import Roboflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2

In [ ]:
rf = Roboflow(api_key="hiXB6MTQOH0hlxROddmK")
project = rf.workspace("jeevan-sendur-g-workspace").project("board-kzxer")
version = project.version(1)
dataset = version.download("yolov8")

In [ ]:
# Load Dataset (Images and Annotations)
def load_data(dataset_path):
    images = []
    labels = []
    for image_file in dataset_path.glob("train/*.jpg"):
        # Load image
        img = cv2.imread(str(image_file))
        img = cv2.resize(img, (224, 224))  # Resize to model input size
        images.append(img)
        
        # Load corresponding label
        label_file = image_file.with_suffix(".txt")
        with open(label_file, "r") as f:
            label = f.read().strip()
            labels.append(np.array([float(x) for x in label.split()]))
    
    return np.array(images), np.array(labels)

# Prepare the dataset
train_images, train_labels = load_data(dataset.location / "train")
test_images, test_labels = load_data(dataset.location / "test")

# Normalize images
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define a simple CNN for keypoint detection
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(8)  # Assuming 4 keypoints, each with (x, y)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [ ]:
# Train the model
model.fit(train_images, train_labels, epochs=20, validation_data=(test_images, test_labels))

In [ ]:
# Save the model
model.save("chessboard_edge_detection_model.h5")